In [1]:
# Reading/Writing Data
import os
import glob
import numpy as np
import math 

# Pytorch
import torch 
from torch.utils.data import Subset, DataLoader

# Self-Defined Package
from SMAPDataset import SMAPDataset
from SDLoss import physics_loss
from Preprocessing import random_spatial_sequence, collate_fn

# Some Utility Functions

In [2]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Configurations
`config` contains hyper-parameters for training and the path to save your model.

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'test_ratio': 0,
    'root': 'D:\\1GRADUATED\\paper\\downscaling_data\\Soil_moisture_downscale_czt\\DATASET\\'
}

# Dataloader
Read data from files and set up training, validation, and testing sets.

In [4]:
# # Set seed for reproducibility
same_seed(config['seed'])

In [5]:
train_sequence = {}
test_sequence = {}
s2s_dir = config['root'] + 'LABEL\\SMAPID2INSITUID'
subdir_list = sorted(os.listdir(s2s_dir))

# 遍历指定路径下的所有子目录
for subdir in subdir_list:
    subdir_path = os.path.join(s2s_dir, subdir)
    if os.path.isdir(subdir_path):
        # 如果子目录是目录而不是文件，则将子目录加入字典
        full_spatial_sequence_smap = sorted([int(f.split('.')[0]) for f in os.listdir(subdir_path) if f.endswith('.npy')]) # !!!! read out of order
        print(len(full_spatial_sequence_smap), 'of Full Spatial Sequence for', subdir, ':')
        print(full_spatial_sequence_smap)
        
        # Split the train\valid\test dataset by spatial dimension
        train_sequence[subdir], test_sequence[subdir] = random_spatial_sequence(1-config['test_ratio'], full_spatial_sequence_smap)

10 of Full Spatial Sequence for 2015104 :
[9, 15, 17, 18, 20, 26, 29, 31, 40, 43]
**************************Data Spliting***************************
Spliting Rate:  1
10 of Dataset1:  [15 31 18 40 29 17 43 26 20  9]
0 of Dataset2:  []
**************************Data Spliting***************************
6 of Full Spatial Sequence for 2015106 :
[15, 16, 20, 26, 27, 29]
**************************Data Spliting***************************
Spliting Rate:  1
6 of Dataset1:  [26 20 27 29 15 16]
0 of Dataset2:  []
**************************Data Spliting***************************
4 of Full Spatial Sequence for 2015114 :
[9, 33, 44, 51]
**************************Data Spliting***************************
Spliting Rate:  1
4 of Dataset1:  [33 51  9 44]
0 of Dataset2:  []
**************************Data Spliting***************************
14 of Full Spatial Sequence for 2015115 :
[15, 16, 17, 18, 20, 26, 29, 31, 32, 33, 40, 41, 43, 44]
**************************Data Spliting***************************
S

In [6]:
# Initialize the dataset
train_dataset = SMAPDataset(config['root'], train_sequence)

***************************Load data path******************************
_______________________________2015104_______________________________
_____________________________smap cell: 15_____________________________
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\SMAP\2015104\15.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\TEXTURE\15.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015104\4.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015104\4.npy
_____________________________smap cell: 31_____________________________
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\SMAP\2015104\31.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\TEXTURE\31.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015104\10.npy
D:\1GRADUATED\paper\downscaling_data\S

D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015131\31.npy
_____________________________smap cell: 41_____________________________
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\SMAP\2015131\41.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\TEXTURE\41.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015131\6.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015131\6.npy
_____________________________smap cell: 10_____________________________
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\SMAP\2015131\10.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\TEXTURE\10.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015131\16.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI

D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015165\54.npy
_____________________________smap cell: 9_____________________________
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\SMAP\2015165\9.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\TEXTURE\9.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015165\55.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015165\55.npy
_____________________________smap cell: 45_____________________________
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\SMAP\2015165\45.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\TEXTURE\45.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015165\37.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\

D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\SMAP\2015226\26.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\TEXTURE\26.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015226\40.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015226\40.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015226\25.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015226\25.npy
_____________________________smap cell: 10_____________________________
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\SMAP\2015226\10.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\TEXTURE\10.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015226\16.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_down

D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015267\40.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015267\40.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015267\25.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015267\25.npy
_____________________________smap cell: 15_____________________________
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\SMAP\2015267\15.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\TEXTURE\15.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015267\4.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015267\4.npy
_______________________________2015272_______________________________
_____________________________smap cell: 9_____________________________
D:\1GRADUAT

D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015330\1.npy
_____________________________smap cell: 33_____________________________
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\SMAP\2015330\33.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\TEXTURE\33.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015330\32.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\ATI\2015330\32.npy
_____________________________smap cell: 31_____________________________
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\SMAP\2015330\31.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\TEXTURE\31.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\SM\2015330\10.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\LABEL\AT

In [7]:
train_dataset.__len__()

409

In [8]:
process_data = [x['processed_data'] for x in train_dataset]
label_data = [x['label_data'] for x in train_dataset]
meta_data = [x['meta_data'] for x in train_dataset]

# Dataset Checking Manually

In [9]:
date = [x['date'] for x in meta_data]
smapid = [x['smapid'] for x in meta_data]
insituid = [x['insituid'] for x in meta_data]

In [10]:
import random

start = 1
end = 409
num_samples = 1

samples = np.random.randint(start, end+1, size=num_samples)
print(samples)

[316]


In [11]:
import datetime

# The index of day start from 1
# day 从 1 开始
def date2day(date, fm="%Y-%m-%d"):
    date_object = datetime.datetime.strptime(date, fm)
    day_of_year = date_object.timetuple().tm_yday
    return day_of_year

def day2date(year, day):
    date = datetime.datetime(year, 1, 1) + datetime.timedelta(day - 1)
    month = date.month
    day = date.day
    return date

In [12]:
for i in samples:
    day = date[i].split('2015')[-1]
    year = date[i].split(day)[0]
    _date = day2date(int(year), int(day)).strftime("%Y-%m-%d")
    print('日期：{} {}'.format(_date, date[i]))
    print('SMAPID：{}'.format(smapid[i]))
    print('smap_norm：{}'.format([process_data[i][0][0][0]]))
    print('len of list：{} {}'.format(len(insituid[i]), len(label_data[i])))
    for j in range(len(insituid[i])):
        print('站点ID：{}'.format(insituid[i][j]))
        print('sm：{}'.format(label_data[i][j][0]))
        print('smap_unorm：{}'.format(label_data[i][j][1]))
        print('ati, atim, atisd：{}'.format(label_data[i][j][2]))

日期：2015-10-12 2015285
SMAPID：15
smap_norm：[0.41653692722320557]
len of list：1 1
站点ID：4
sm：[0.19586364]
smap_unorm：[0.21075247]
ati, atim, atisd：[0.0665961465621 0.052126798778772354 0.010095000267028809]


In [13]:
root = 'D:\\1GRADUATED\\paper\\downscaling_data\\Soil_moisture_downscale_czt\\DATASET\\LABEL\\SMAP\\2015285\\'

In [14]:
arr = np.load(root+'15.npy')
print(arr)

[0.21075247]


In [15]:
root = 'D:\\1GRADUATED\\paper\\downscaling_data\\Soil_moisture_downscale_czt\\DATASET\\LABEL\\ATI\\2015285\\'

In [16]:
arr = np.load(root+'4.npy', allow_pickle=True)
print(arr)

[0.0665961465621 0.052126798778772354 0.010095000267028809]


In [21]:
print(date2day('2015-09-01'))

244


## 经过检验：
> - dataset读取的数据与文件夹里的相同
> - 文件夹里的数据（smap，ati）与arcmap对齐后提取的excel里的数据相同
> - 提取的excel与原始tiff相同

## 检查smap与texture是否对齐

In [18]:
x = 2
y = 2
for i in samples:
    print(process_data[i][2, 2, 0])
    print(process_data[i][2, 2, 1])
    print(process_data[i][2, 2, 2])

0.41653692722320557
13.0
56.0


# Correlation Analysis